In [4]:
import json
import requests
from datetime import datetime

ELASTIC = "http://localhost:9200" #dirección en la que está funcionando elastic

In [312]:
"""
    Reservas de duración mayor o igual a 7 días
"""

print("Reservas de duración igual o superior a una semana:")
print("")

query = {
    "query":
    {
        "script" : {
            "script" : {
                "source": "return (doc['CheckOut'].value.getMillis() - doc['CheckIn'].value.getMillis())/(1000*60*60*24) >= 7"
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["hits"]["hits"]

for r in res:
    reservation = r["_source"]
    ckin = datetime.strptime(reservation["CheckIn"], "%d-%b-%y")
    ckout = datetime.strptime(reservation["CheckOut"], "%d-%b-%y")

    print("%s %s: Entró %s, Salió %s, duración = %d días" % (reservation["FirstName"], reservation["LastName"],reservation["CheckIn"],reservation["CheckOut"],(ckout-ckin).days))

print("")


Reservas de duración igual o superior a una semana:

JESSICA SULOUFF: Entró 26-Feb-10, Salió 07-Mar-10, duración = 9 días
EDWARDO CHINAULT: Entró 21-Mar-10, Salió 28-Mar-10, duración = 7 días
STANFORD WAEGNER: Entró 21-Apr-10, Salió 30-Apr-10, duración = 9 días
JONATHON BURCHAM: Entró 06-May-10, Salió 13-May-10, duración = 7 días
HONEY FOCKE: Entró 30-May-10, Salió 06-Jun-10, duración = 7 días
LEWIS TRUDEN: Entró 21-Jun-10, Salió 02-Jul-10, duración = 11 días
NATHANAEL HARTFORD: Entró 03-Sep-10, Salió 10-Sep-10, duración = 7 días
GUS DERKAS: Entró 01-Oct-10, Salió 13-Oct-10, duración = 12 días
OLYMPIA ALBROUGH: Entró 14-Oct-10, Salió 25-Oct-10, duración = 11 días
HERMAN KUTA: Entró 25-Oct-10, Salió 01-Nov-10, duración = 7 días



In [311]:
"""
    Personas que se han alojado más de una vez
"""

print("Personas que se han alojado más de una vez:")
print("")

query = {
    "size":0,
    "aggs": {
        "persona":{
            "terms" : {
                "script": "params['_source'].FirstName+' '+params['_source'].LastName"
            },
            "aggs":{
                "reservas":{
                    "bucket_selector": {
                        "buckets_path": {
                            "cuenta" : "_count"
                        },
                        "script": {
                            "source" : "params.cuenta > 1"
                        }
                    }
                }
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["aggregations"]["persona"]["buckets"]

for r in res:
    print("%s se ha alojado %d veces" % (r["key"], r["doc_count"]))

Personas que se han alojado más de una vez:

EMERY VOLANTE se ha alojado 3 veces
GLEN DONIGAN se ha alojado 3 veces
GRANT KNERIEN se ha alojado 3 veces
ALEXIS FINEFROCK se ha alojado 2 veces
AMIEE PENEZ se ha alojado 2 veces
ARON KEBEDE se ha alojado 2 veces
AUGUST MAEWEATHER se ha alojado 2 veces
BENTON AKHTAR se ha alojado 2 veces
BO DURAN se ha alojado 2 veces
CATHERIN KUDRON se ha alojado 2 veces


In [307]:
"""
    Número de reservas al mes para camas de tipo Queen
"""

print("Número de reservas al mes para camas de tipo Queen:")
print("")

query = {
    "size" : 0,
    "query" : {
        "match" : { "Room.bedType" : "Queen" }
    },
    "aggs" : {
        "reservas-al-mes" : {
            "terms" : { 
                "script" : "doc['CheckIn'].value.getMonth()"
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["aggregations"]["reservas-al-mes"]["buckets"]

for r in res:
    print("En el mes %s: %d reservas de camas de tipo Queen" % (r["key"],r["doc_count"]))


Número de reservas al mes para camas de tipo Queen:

En el mes AUGUST: 25 reservas de camas de tipo Queen
En el mes DECEMBER: 24 reservas de camas de tipo Queen
En el mes APRIL: 21 reservas de camas de tipo Queen
En el mes JULY: 21 reservas de camas de tipo Queen
En el mes OCTOBER: 21 reservas de camas de tipo Queen
En el mes NOVEMBER: 19 reservas de camas de tipo Queen
En el mes SEPTEMBER: 19 reservas de camas de tipo Queen
En el mes JUNE: 18 reservas de camas de tipo Queen
En el mes MAY: 18 reservas de camas de tipo Queen
En el mes JANUARY: 16 reservas de camas de tipo Queen


In [338]:
"""
    Precio total de las reservas realizadas por EMERY VOLANTE
"""

query = {
    "query" : {
        "bool" : {
            "must" : [{"match" : { "FirstName" : "EMERY"}}, {"match" : { "LastName" : "VOLANTE"}}]
        }
    },
    "aggs" : {
       "precio-total" : {
           "sum" : {
               "script" : {
                   "source" : "(doc.CheckOut.value.getMillis() - doc.CheckIn.value.getMillis())/(1000*60*60*24) * doc.Rate.value" 
               }
           }
       } 
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["aggregations"]

print("Precio total de todas las reservas realizadas por EMERY VOLANTE: $%0.2f" % res["precio-total"]["value"])
print("")

Precio total de todas las reservas realizadas por EMERY VOLANTE: $843.75

